In [1]:
# import os
import librosa

import numpy as np
import pandas as pd
import seaborn as sns
# import tensorflow as tf

import librosa.display

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline


C:\Users\sukal\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\sukal\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\sukal\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [3]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the default MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=13, frame_length_ms=25, hop_length_ms=10)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_default1.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

C:\Users\sukal\anaconda3\Lib\site-packages\librosa\feature\spectral.py:2148: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Processing file 212/241 (87.97%) - 358-0.wav
Processing file 213/241 (88.38%) - 358-1.wav
Processing file 214/241 (88.80%) - 360-0.wav
Processing file 215/241 (89.21%) - 361-0.wav
Processing file 216/241 (89.63%) - 362-0.wav
Processing file 217/241 (90.04%) - 362-1.wav
Processing file 218/241 (90.46%) - 368-0.wav
Processing file 219/241 (90.87%) - 369-0.wav
Processing file 220/241 (91.29%) - 381-0.wav
Processing file 221/241 (91.70%) - 381-1.wav
Processing file 222/241 (92.12%) - 450-0.wav
Processing file 223/241 (92.53%) - 450-1.wav
Processing file 224/241 (92.95%) - 458-0.wav
Processing file 225/241 (93.36%) - 461-0.wav
Processing file 226/241 (93.78%) - 465-0.wav
Processing file 227/241 (94.19%) - 466-0.wav
Processing file 228/241 (94.61%) - 466-1.wav
Processing file 229/241 (95.02%) - 468-0.wav
Processing file 230/241 (95.44%) - 470-1.wav
Processing file 231/241 (95.85%) - 471-0.wav
Processing file 232/241 (96.27%) - 472-0.wav
Processing file 233/241 (96.68%) - 474-0.wav
Processing

In [7]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the default MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=20, frame_length_ms=25, hop_length_ms=10)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_default2.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

In [9]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the best MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=20, frame_length_ms=25, hop_length_ms=5)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_best1.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

In [10]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the best MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=20, frame_length_ms=25, hop_length_ms=50)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_best2.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

In [11]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the worst MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=30, frame_length_ms=100, hop_length_ms=30)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_worst1.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

In [12]:
import librosa
import numpy as np
import pandas as pd
import os

# Function to extract mean MFCC features
def extract_mean_mfcc(audio_file, n_mfcc=13, frame_length_ms=25, hop_length_ms=10):
    """
    Extracts MFCC features from an audio file and computes the mean across frames.
    Returns the mean MFCC features as a list.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Compute hop length and FFT window size
    hop_length = int((hop_length_ms / 1000) * sr)  
    n_fft = int((frame_length_ms / 1000) * sr)    

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)

    # Compute the mean of MFCCs across time frames
    mean_mfccs = np.mean(mfccs, axis=1)

    return mean_mfccs

# Function to process audio files in multiple folders
def process_audio_folders(input_folders, output_directory="output_csvs"):
    """
    Processes all audio files in multiple input folders for the worst MFCC parameters,
    extracts mean MFCCs, and saves results to a single CSV.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # List to store results
    mfcc_results = []

    # Process each folder
    for folder in input_folders:
        # Get all audio files in the folder
        audio_files = [f for f in os.listdir(folder) if f.endswith(('.wav', '.mp3'))]
        total_files = len(audio_files)
        
        if total_files == 0:
            print(f"No audio files found in folder: {folder}!")
            continue
        
        # Process each audio file in the folder
        print(f"\nProcessing {total_files} files in folder: {folder}")
        for i, audio_file in enumerate(audio_files):
            audio_path = os.path.join(folder, audio_file)
            
            # For each audio file, process with the default parameters
            mean_mfccs = extract_mean_mfcc(audio_path, n_mfcc=30, frame_length_ms=50, hop_length_ms=80)
            
            # Append results with filename, folder, and mean MFCC values
            mfcc_results.append([audio_file, folder] + mean_mfccs.tolist())

            # Output progress in the console for the current file
            print(f"Processing file {i + 1}/{total_files} ({(i + 1) / total_files * 100:.2f}%) - {audio_file}")
        
    # Convert to DataFrame
    column_names = ["Filename", "Folder"] + [f"MFCC_{i+1}" for i in range(len(mean_mfccs))]
    mfcc_df = pd.DataFrame(mfcc_results, columns=column_names)

    # Save the DataFrame to a CSV file
    output_csv = os.path.join(output_directory, "mfcc_means_worst2.csv")
    mfcc_df.to_csv(output_csv, index=False)
    
    # Print success message after saving the CSV
    print(f"\n✅ Mean MFCCs saved to {output_csv}")

# Define input folders containing audio files
input_folders = [
    r"C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia",  # First folder
    r"C:\Users\sukal\Downloads\Speech Sample\Control\Control"  # Second folder (replace with the second folder path)
]

# Call the function to process the files and save the results to a CSV file
process_audio_folders(input_folders)



Processing 241 files in folder: C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia
Processing file 1/241 (0.41%) - 001-0.wav
Processing file 2/241 (0.83%) - 001-2.wav
Processing file 3/241 (1.24%) - 003-0.wav
Processing file 4/241 (1.66%) - 005-0.wav
Processing file 5/241 (2.07%) - 005-2.wav
Processing file 6/241 (2.49%) - 007-1.wav
Processing file 7/241 (2.90%) - 007-3.wav
Processing file 8/241 (3.32%) - 010-0.wav
Processing file 9/241 (3.73%) - 010-1.wav
Processing file 10/241 (4.15%) - 010-2.wav
Processing file 11/241 (4.56%) - 010-3.wav
Processing file 12/241 (4.98%) - 010-4.wav
Processing file 13/241 (5.39%) - 014-2.wav
Processing file 14/241 (5.81%) - 016-0.wav
Processing file 15/241 (6.22%) - 016-1.wav
Processing file 16/241 (6.64%) - 016-3.wav
Processing file 17/241 (7.05%) - 016-4.wav
Processing file 18/241 (7.47%) - 018-0.wav
Processing file 19/241 (7.88%) - 023-0.wav
Processing file 20/241 (8.30%) - 023-2.wav
Processing file 21/241 (8.71%) - 024-1.wav
Processing file 

In [31]:
import os
import pandas as pd

# Specify the folder where the CSV files are located
folder_path = r'C:\Users\sukal\output_csvs'  # Replace this with your folder path

# Get the list of CSV files in the specified folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Loop through each file
for file in csv_files:
    file_path = os.path.join(folder_path, file)  # Full path of the file
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Binary encode the 'Folder' column
    df['Folder'] = df['Folder'].map({
        r'C:\Users\sukal\Downloads\Speech Sample\Dementia\Dementia': 1,
        r'C:\Users\sukal\Downloads\Speech Sample\Control\Control': 0
    })
    
    # Create a new filename for the modified file
    new_file = file.replace('.csv', '_SVM.csv')
    new_file_path = os.path.join(r'C:\Users\sukal\output_csvs_SVM', new_file)  # Full path for saving
    
    # Save the modified DataFrame to a new CSV file
    df.to_csv(new_file_path, index=False)
    
    # Print a message indicating the file was processed
    print(f"Column dropped and binary encoding done successfully for {file}, saved as {new_file}.")
    

Column dropped and binary encoding done successfully for mfcc_means_best1 (20, 25, 5).csv, saved as mfcc_means_best1 (20, 25, 5)_SVM.csv.
Column dropped and binary encoding done successfully for mfcc_means_best2 (20, 25, 50).csv, saved as mfcc_means_best2 (20, 25, 50)_SVM.csv.
Column dropped and binary encoding done successfully for mfcc_means_default1 (13, 25, 10).csv, saved as mfcc_means_default1 (13, 25, 10)_SVM.csv.
Column dropped and binary encoding done successfully for mfcc_means_default2 (20, 25, 10).csv, saved as mfcc_means_default2 (20, 25, 10)_SVM.csv.
Column dropped and binary encoding done successfully for mfcc_means_worst1 (30, 100, 30).csv, saved as mfcc_means_worst1 (30, 100, 30)_SVM.csv.
Column dropped and binary encoding done successfully for mfcc_means_worst2 (30, 50, 80).csv, saved as mfcc_means_worst2 (30, 50, 80)_SVM.csv.


In [77]:
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Folder containing the CSV files
folder_path = r"C:\Users\sukal\output_csvs_SVM"

# Iterate through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Read CSV
        df = pd.read_csv(file_path)

        df=df.drop(columns=['Filename'])  # Drop target column
        
        # Ensure the target column 'output' exists
        if 'Folder' not in df.columns:
            print(f"Skipping {filename} (No 'Folder' column)")
            continue
        
        # Separate features (X) and target variable (y)
        X = df.drop(columns=['Folder'])  # Drop target column
        y = df['Folder']

        # Handle missing values (fill with mean)
        # X = X.fillna(X.mean())

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

        # Train SVM model
        model = SVC(kernel='poly')  # Use RBF kernel for non-linear classification
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"SVM Model with poly kernel and test ratio 0.1 trained on {filename} - Accuracy: {accuracy:.4f}")

        # Train KNN Model
        modelK = KNeighborsClassifier(n_neighbors=9)
        modelK.fit(X_train, y_train)

        # Predict and evaluate
        y_predK = modelK.predict(X_test)
        accuracyK = accuracy_score(y_test, y_predK)

        # Print the accuracy for each file
        print(f"KNN Model with K-value of 9 and test ratio 0.1 trained on {filename} - Accuracy: {accuracyK:.4f}\n")

print("\nSVM and KNN applied to all CSV files successfully!")

SVM Model with poly kernel and test ratio 0.1 trained on mfcc_means_best1 (20, 25, 5)_SVM.csv - Accuracy: 0.6327
KNN Model with K-value of 9 and test ratio 0.1 trained on mfcc_means_best1 (20, 25, 5)_SVM.csv - Accuracy: 0.6122

SVM Model with poly kernel and test ratio 0.1 trained on mfcc_means_best2 (20, 25, 50)_SVM.csv - Accuracy: 0.5714
KNN Model with K-value of 9 and test ratio 0.1 trained on mfcc_means_best2 (20, 25, 50)_SVM.csv - Accuracy: 0.6531

SVM Model with poly kernel and test ratio 0.1 trained on mfcc_means_default1 (13, 25, 10)_SVM.csv - Accuracy: 0.5510
KNN Model with K-value of 9 and test ratio 0.1 trained on mfcc_means_default1 (13, 25, 10)_SVM.csv - Accuracy: 0.4898

SVM Model with poly kernel and test ratio 0.1 trained on mfcc_means_default2 (20, 25, 10)_SVM.csv - Accuracy: 0.6327
KNN Model with K-value of 9 and test ratio 0.1 trained on mfcc_means_default2 (20, 25, 10)_SVM.csv - Accuracy: 0.6122

SVM Model with poly kernel and test ratio 0.1 trained on mfcc_means_wo